In [1]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io
import shutil

from __future__ import print_function

import os.path

In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/documents.readonly', 'https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.activity.readonly']

# The ID of a sample document.
DOCUMENT_ID = '17sfw7Ifjxh6h5Tina0dM--8AAA-ZTefyQVc_7vwyVXo' # take from environment variable

check validity of token.json and when it expires and how it refreshes after expiry

specific access to provided emails and also public access


one file entry function 
- will get points to be added to summary
- will get points to be added to experience
- will get points to be added to languages

create different variables for these texts

### Get credentials

In [3]:
def main():
    """Shows basic usage of the Docs API.
    Prints the title of a sample document.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds

creds=main()

### Get document

In [4]:
try:
    service = build('docs', 'v1', credentials=creds) #build('docs', 'v1', developerKey=API_KEY) 

    # Retrieve the documents contents from the Docs service.
    document = service.documents().get(documentId=DOCUMENT_ID).execute()

    print('The title of the document is: {}'.format(document.get('title')))
    # print(document.get('body'))
            
except HttpError as err:
    print(err)

The title of the document is: Copy of Raj resue


### Find placeholders

In [5]:
for i in document.get('body')['content']:
    try:
        if 'bullet' in i['paragraph'].keys():
            print(i)
    except:
        continue

{'startIndex': 575, 'endIndex': 639, 'paragraph': {'elements': [{'startIndex': 575, 'endIndex': 639, 'textRun': {'content': 'Recognized by the ICSE Board for achieving a 94.8% in X Boards.\n', 'textStyle': {'bold': False, 'italic': False, 'underline': False, 'strikethrough': False, 'smallCaps': False, 'backgroundColor': {}, 'foregroundColor': {'color': {'rgbColor': {}}}, 'fontSize': {'magnitude': 10, 'unit': 'PT'}, 'weightedFontFamily': {'fontFamily': 'Times New Roman', 'weight': 400}, 'baselineOffset': 'NONE'}}}], 'paragraphStyle': {'namedStyleType': 'NORMAL_TEXT', 'alignment': 'START', 'lineSpacing': 100, 'direction': 'LEFT_TO_RIGHT', 'spacingMode': 'NEVER_COLLAPSE', 'spaceAbove': {'unit': 'PT'}, 'spaceBelow': {'unit': 'PT'}, 'borderBetween': {'color': {}, 'width': {'unit': 'PT'}, 'padding': {'unit': 'PT'}, 'dashStyle': 'SOLID'}, 'borderTop': {'color': {}, 'width': {'unit': 'PT'}, 'padding': {'unit': 'PT'}, 'dashStyle': 'SOLID'}, 'borderBottom': {'color': {}, 'width': {'unit': 'PT'},

In [18]:
for i in document.get('body')['content']:
    if 'paragraph' in i.keys():
        try:
            for j in i['paragraph']['elements']:
                print(j['textRun']['content'], i['paragraph']['elements'])
        except Exception as e:
            print(e)

Raj Kiran                                                                                                                                                        [{'startIndex': 1, 'endIndex': 161, 'textRun': {'content': 'Raj Kiran                                                                                                                                                       ', 'textStyle': {'bold': True, 'italic': False, 'underline': False, 'strikethrough': False, 'smallCaps': False, 'backgroundColor': {}, 'foregroundColor': {'color': {'rgbColor': {}}}, 'fontSize': {'magnitude': 11, 'unit': 'PT'}, 'weightedFontFamily': {'fontFamily': 'Arial Narrow', 'weight': 400}, 'baselineOffset': 'NONE'}}}, {'startIndex': 161, 'endIndex': 162, 'inlineObjectElement': {'inlineObjectId': 'i.0', 'textStyle': {'bold': True, 'italic': False, 'underline': False, 'strikethrough': False, 'smallCaps': False, 'backgroundColor': {}, 'foregroundColor': {'color': {'rgbColor': {}}}, 'fontSize': {'magnitude': 1

In [34]:
for i in document.get('body')['content']:
    # print(i.keys())
    if 'paragraph' in i.keys():
        try:
            for j in i['paragraph']['elements']:
                  if '#' in j['textRun']['content']:
                    print(j['textRun']['content'], i['paragraph']['elements'])
        except Exception as e:
            print(e)

    if 'table' in i.keys():
        for j in i['table']['tableRows']:
            text_val=j['tableCells'][1]['content'][0]['paragraph']['elements'][0]['textRun']['content']
            if '# LANGUAGES' in  text_val:
                print(text_val, j['tableCells'][1]['content'][0]['paragraph']['elements'][0])

'textRun'
# ABOUT ME
 [{'startIndex': 324, 'endIndex': 335, 'textRun': {'content': '# ABOUT ME\n', 'textStyle': {}}}]
# SUMMARY POINTS [{'startIndex': 365, 'endIndex': 381, 'textRun': {'content': '# SUMMARY POINTS', 'textStyle': {}}}, {'startIndex': 381, 'endIndex': 382, 'textRun': {'content': '\n', 'textStyle': {'weightedFontFamily': {'fontFamily': 'Trebuchet MS', 'weight': 400}}}}]
# LANGUAGES {'startIndex': 1375, 'endIndex': 1386, 'textRun': {'content': '# LANGUAGES', 'textStyle': {'fontSize': {'magnitude': 11, 'unit': 'PT'}, 'weightedFontFamily': {'fontFamily': 'Arial Narrow', 'weight': 400}}}}
# RESPONSIBILITIES 1 [{'startIndex': 6758, 'endIndex': 6778, 'textRun': {'content': '# RESPONSIBILITIES 1', 'textStyle': {}}}, {'startIndex': 6778, 'endIndex': 6779, 'textRun': {'content': '\n', 'textStyle': {'bold': True}}}]
# RESPONSIBILITIES 2
 [{'startIndex': 14371, 'endIndex': 14392, 'textRun': {'content': '# RESPONSIBILITIES 2\n', 'textStyle': {}}}]


### Store document content as json. (data is originally a dictionary)

In [20]:
import json

with open('data.json', 'w') as fp:
    json.dump(document.get('body'), fp)

### Perform manipulations directly to google doc

In [31]:
requests = [
    ### removing bullet points
    {
        'deleteParagraphBullets': {
            'range': {
                'startIndex': 6758,
                'endIndex':  6778
            },
        }
    },
    ### adding text (last point endIndex)
    # {
    #     'insertText': {
    #         'location': {
    #             'index': 1410,
    #         },
    #         'text': 'abcdefg \n'
    #     }
    # },
    ### deleting text (startIndex-1)
    {
            'deleteContentRange': {
                'range': {
                    'startIndex': 6758,
                    'endIndex': 6777,
                }

            }

    },
    ## creating bullet points
    # {
    #     'createParagraphBullets': {
    #         'range': {
    #             'startIndex': 1410,
    #             'endIndex':  1410
    #         },
    #         'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE',
    #     }
    # },
    

]

try:
    service = build('docs', 'v1', credentials=creds)
    result = service.documents().batchUpdate(
        documentId=DOCUMENT_ID, body={'requests': requests}).execute()
except HttpError as e:
    print(e)

### Duplicate a file

In [38]:
# Build the Google Drive API client
drive_service = build('drive', 'v3', credentials=creds)

# ID of the file you want to duplicate
source_file_id = DOCUMENT_ID

# Define the new name for the duplicated file
new_name = 'Dupl resume'

# Create a copy of the file
duplicate_file = {
    'name': new_name
}

drive_service.files().copy(fileId=source_file_id, body=duplicate_file).execute()

print(f'{new_name} duplicated successfully.')

Dupl resume duplicated successfully.


### List files in drive account

In [41]:
# List files in your Google Drive
results = drive_service.files().list().execute()
files = results.get('files', [])

if not files:
    print('No files found.')
else:
    print('Files:')
    for file in files:
        if file['name']==new_name:
            print(f"{file['name']} ({file['id']})")

Files:
Dupl resume (18KMdjul-S2m5LmUHdNWk7_jLpLpW2tTJTyLSmawGyJI)


### Dowwnload file locally

In [37]:
def download_file(real_file_id, creds, filename):
    """Downloads a file
    Args:
        real_file_id: ID of the file to download
    Returns : IO object with location.

    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.
    """
    try:
        # create drive api client
        service = build('drive', 'v3', credentials=creds)

        file_id = real_file_id

        # pylint: disable=maybe-no-member
        request = service.files().export_media(fileId=file_id, mimeType='application/pdf') # 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            # print(F'Download {int(status.progress() * 100)}. Done: {done}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    print(file)
    file.seek(0)
    with open(filename, 'wb') as f:
        shutil.copyfileobj(file, f)

    # return file

# https://docs.google.com/document/d/17sfw7Ifjxh6h5Tina0dM--8AAA-ZTefyQVc_7vwyVXo/edit?usp=drive_link
download_file(real_file_id=DOCUMENT_ID, creds=creds, filename='./downloaded.pdf')

### Upload files to drive

In [33]:
def uploadFile(filename, uploadname):
    service = build('drive', 'v3', credentials=creds)

    file_metadata = {
    'name': uploadname,
    'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
    }
    media = MediaFileUpload(filename ,
                            mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document',
                            resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print ('File ID: ' + file.get('id'))

uploadFile('downloaded.docx', 'uploaded.docx')

File ID: 1Ejtv9pa8XDg5I4wnvuqW4e-inyXelfoB


### Drive Activity API

In [4]:
# Returns the name of a set property in an object, or else "unknown".
def getOneOf(obj):
    for key in obj:
        return key
    return 'unknown'

# Returns a time associated with an activity.
def getTimeInfo(activity):
    if 'timestamp' in activity:
        return activity['timestamp']
    if 'timeRange' in activity:
        return activity['timeRange']['endTime']
    return 'unknown'

# Returns the type of action.
def getActionInfo(actionDetail):
    return getOneOf(actionDetail)

# Returns user information, or the type of user if not a known user.
def getUserInfo(user):
    if 'knownUser' in user:
        knownUser = user['knownUser']
        isMe = knownUser.get('isCurrentUser', False)
        return u'people/me' if isMe else knownUser['personName']
    return getOneOf(user)

# Returns actor information, or the type of actor if not a user.
def getActorInfo(actor):
    if 'user' in actor:
        return getUserInfo(actor['user'])
    return getOneOf(actor)

# Returns the type of a target and an associated title.
def getTargetInfo(target):
    if 'driveItem' in target:
        title = target['driveItem'].get('title', 'unknown')
        return 'driveItem:"{0}"'.format(title)
    if 'drive' in target:
        title = target['drive'].get('title', 'unknown')
        return 'drive:"{0}"'.format(title)
    if 'fileComment' in target:
        parent = target['fileComment'].get('parent', {})
        title = parent.get('title', 'unknown')
        return 'fileComment:"{0}"'.format(title)
    return '{0}:unknown'.format(getOneOf(target))

In [6]:
# Call the Drive Activity API
try:
    activity_service = build('driveactivity', 'v2', credentials=creds)
    results = activity_service.activity().query(body={
        "itemName": f"items/{DOCUMENT_ID}",
        'pageSize': 10
    }).execute()
    activities = results.get('activities', [])

    if not activities:
        print('No activity.')
    else:
        print('Recent activity:')
        for activity in activities:
            time = getTimeInfo(activity)
            action = getActionInfo(activity['primaryActionDetail'])
            actors = map(getActorInfo, activity['actors'])
            targets = map(getTargetInfo, activity['targets'])
            actors_str, targets_str = "", ""
            actor_name = actors_str.join(actors)
            target_name = targets_str.join(targets)

            # Print the action occurred on drive with actor, target item and timestamp
            print(u'{0}: {1}, {2}, {3}'.format(time, action, actor_name, target_name))

except HttpError as error:
    # TODO(developer) - Handleerrors from drive activity API.
    print(f'An error occurred: {error}')

Recent activity:
2023-08-17T18:31:02.620Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-17T18:23:59.011Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-17T18:13:20.944Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-17T18:07:30.532Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-17T18:07:18.075Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-15T16:14:24.914Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-15T16:11:23.230Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-15T16:04:41.673Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-15T16:02:02.654Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-15T16:01:40.128Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-14T13:33:07.846Z: edit, people/me, driveItem:"J017_Yash_Chauhan_MPSTME_new"
2023-08-14T13:30:06.492Z: edit, people/me, driveItem:"J017_

In [13]:
service = build('drive', 'v3', credentials=creds)

revised=service.revisions().list(fileId=DOCUMENT_ID).execute()

for i in revised['revisions']:
    print(u'{0}: {1}, {2}, {3}'.format(i['id'], i['mimeType'], i['kind'], i['modifiedTime']))

1: application/vnd.google-apps.document, drive#revision, 2023-08-14T09:40:15.108Z
25: application/vnd.google-apps.document, drive#revision, 2023-08-14T10:14:05.219Z
77: application/vnd.google-apps.document, drive#revision, 2023-08-14T13:30:56.057Z
110: application/vnd.google-apps.document, drive#revision, 2023-08-15T16:12:10.851Z
124: application/vnd.google-apps.document, drive#revision, 2023-08-17T18:23:57.715Z


### Permissions

In [16]:
### List all permissions
try:
    service = build('drive', 'v3', credentials=creds)

    print(service.permissions().list(fileId=DOCUMENT_ID).execute())
except HttpError as e:
    print(e)

{'kind': 'drive#permissionList', 'permissions': [{'id': '08554114178509853196', 'type': 'user', 'kind': 'drive#permission', 'role': 'owner'}]}


In [10]:
### delete a permission
try:
    service.permissions().delete(fileId=DOCUMENT_ID, permissionId='anyoneWithLink').execute()
except HttpError as e:
    print(e)

<HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/17sfw7Ifjxh6h5Tina0dM--8AAA-ZTefyQVc_7vwyVXo/permissions/anyoneWithLink? returned "Permission not found: anyoneWithLink.". Details: "[{'message': 'Permission not found: anyoneWithLink.', 'domain': 'global', 'reason': 'notFound', 'location': 'permissionId', 'locationType': 'parameter'}]">


In [12]:
# change single permission for user
user_permission = {
            'type': 'user', # domain
            'role': 'writer', # reader/commenter/owner
            'emailAddress': 'yashc1988@gmail.com'
        }

try:
    service.permissions().create(fileId=DOCUMENT_ID,
                                body=user_permission,
                                fields='id',).execute()
except HttpError as error:
        print(F'An error occurred: {error}')

In [15]:
# change domain permission
domain_permission = {
            'type': 'domain',
            'role': 'reader',
            'domain': 'cerebrone.ai'
        }
try:
    service.permissions().create(fileId=DOCUMENT_ID,
                                body=domain_permission,
                                fields='id',).execute()
except HttpError as error:
        print(F'An error occurred: {error}')

In [20]:
### changes in batches
ids=[]
def callback(request_id, response, exception):
    if exception:
        # Handle error
        print(exception)
    else:
        print(f'Request_Id: {request_id}')
        print(F'Permission Id: {response.get("id")}')
        ids.append(response.get('id'))

# pylint: disable=maybe-no-member
batch = service.new_batch_http_request(callback=callback)

user_permission = {
    'type': 'user',
    'role': 'writer',
    'emailAddress': 'yashc1988@gmail.com'
}
batch.add(service.permissions().create(fileId=DOCUMENT_ID,
                                        body=user_permission,
                                        fields='id',))
domain_permission = {
    'type': 'domain',
    'role': 'reader',
    'domain': 'cerebrone.ai'
}
batch.add(service.permissions().create(fileId=DOCUMENT_ID,
                                        body=domain_permission,
                                        fields='id',))
batch.execute()


Request_Id: 1
Permission Id: 07533564481549323021
Request_Id: 2
Permission Id: 01762221233356229123k
